In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten
from keras.utils import image_dataset_from_directory
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications import DenseNet121
from keras.optimizers import Adam

import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

2025-10-19 08:03:35.090841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760861015.289127      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760861015.344767      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data_dir = "/kaggle/input/realwaste/realwaste-main/RealWaste"
img_size = (224, 224)
batch_size = 32

# Create full dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

# Calculate splits
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

# Step 3 — Split
train_ds = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_ds = remaining.take(val_size)
test_ds = remaining.skip(val_size)

Found 4752 files belonging to 9 classes.


In [ ]:
# Load DenseNet Model

base_model = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

base_model.trainable = False # Freeze early layers

In [ ]:
# Dense base model output
inputs = base_model.output

# Custom classification layer
x = Flatten()(inputs)
x = Dense(512, activation='relu')(x)
x = Dense(9, activation='softmax')(x)

# Define full model
dense_model = tf.keras.Model(inputs=base_model.input, outputs=x)

In [ ]:
dense_model.summary()

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.Input(shape=(224,224,3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1, fill_mode='nearest'),
    tf.keras.layers.RandomTranslation(0.1,0.1,fill_mode='nearest'),
    tf.keras.layers.RandomZoom(0.1, fill_mode='nearest'),
])

In [ ]:
dense_model_aug = tf.keras.Sequential([
    data_augmentation,
    dense_model,
])

In [ ]:
es=EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    verbose=1,
    restore_best_weights=True,
)
lr=ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=10,
    verbose=1,
    min_lr=1e-10,
)

In [ ]:
dense_model_aug.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = Adam(learning_rate=0.001),
    metrics = ['accuracy'],
)

In [ ]:
history = dense_model_aug.fit(
    train_ds,
    epochs = 100,
    validation_data = val_ds,
    callbacks = [es, lr]
)

In [ ]:
# Plot training & validation accuracy, loss values
history = dense_model_aug.history.history

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot training & validation accuracy values
ax1.plot(history['accuracy'], label='Training Accuracy')
ax1.plot(history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend()

# Plot training & validation loss values
ax2.plot(history['loss'], label='Training Loss')
ax2.plot(history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend()

plt.savefig('/kaggle/working/accuracy_loss_plot.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# Retrieve best metrics
best_train_acc = max(history['accuracy'])
best_val_acc = max(history['val_accuracy'])
print(f"Best training accuracy is {(best_train_acc * 100):.2f}%")
print(f"Best validation accuracy is {(best_val_acc * 100):.2f}%")